# `DataFrame` Demo (Covid-19)

In this notebook we are going to create a `DataFrame` from data hosted in a [Renkulab project](https://renkulab.io/projects/covid-19/covid-19-public-data) that aggregates public data sources to better understand the spread and effect of covid-19.
The data is refreshed every day. If you are interested, feel free to contribute by adding data sources or trying out some data science! 

We start a Spark application (using sparkmagic).

In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
77,application_1583239045420_3644,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Import and create the `SQLContext`.

In [2]:
from pyspark.sql import Row
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We create a URL variable to get the confirmed cases by country and date. The source of the data is taken from the [COVID-19 data repository by John Hopkings University](https://github.com/CSSEGISandData/COVID-19).
We add the file to the `SparkContext`.

In [3]:
url = "https://renkulab.io/gitlab/covid-19/covid-19-public-data/raw/master/data/covid-19_jhu-csse/time_series_19-covid-Confirmed.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark import SparkFiles
sc.addFile(url)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now we can create the `DataFrame` by reading the csv file, notice that the `Schema` will be inferred automatically from the first row.

In [5]:
df = sqlContext.read.csv("file://"+SparkFiles.get("time_series_19-covid-Confirmed.csv"), header=True, inferSchema= True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- 1/22/20: integer (nullable = true)
 |-- 1/23/20: integer (nullable = true)
 |-- 1/24/20: integer (nullable = true)
 |-- 1/25/20: integer (nullable = true)
 |-- 1/26/20: integer (nullable = true)
 |-- 1/27/20: integer (nullable = true)
 |-- 1/28/20: integer (nullable = true)
 |-- 1/29/20: integer (nullable = true)
 |-- 1/30/20: integer (nullable = true)
 |-- 1/31/20: integer (nullable = true)
 |-- 2/1/20: integer (nullable = true)
 |-- 2/2/20: integer (nullable = true)
 |-- 2/3/20: integer (nullable = true)
 |-- 2/4/20: integer (nullable = true)
 |-- 2/5/20: integer (nullable = true)
 |-- 2/6/20: integer (nullable = true)
 |-- 2/7/20: integer (nullable = true)
 |-- 2/8/20: integer (nullable = true)
 |-- 2/9/20: integer (nullable = true)
 |-- 2/10/20: integer (nullable = true)
 |-- 2/11/20: integer (nullable = true)
 

Now that we know how the `schema` looks like, let's take a peek to the data.

In [14]:
df.select('Province/State', 'Country/Region','3/22/20').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------+-------+
|Province/State|Country/Region|3/22/20|
+--------------+--------------+-------+
|          null|      Thailand|    599|
|          null|         Japan|   1086|
|          null|     Singapore|    455|
|          null|         Nepal|      2|
|          null|      Malaysia|   1306|
+--------------+--------------+-------+
only showing top 5 rows

To be able to perform some actions to the `DataFrame` more easily we can rename the columns of our interest.

In [15]:
df = df.withColumnRenamed("Province/State", "province_state")
df = df.withColumnRenamed("Country/Region", "country_region")
df = df.withColumnRenamed("3/23/20","march23")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- province_state: string (nullable = true)
 |-- country_region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- 1/22/20: integer (nullable = true)
 |-- 1/23/20: integer (nullable = true)
 |-- 1/24/20: integer (nullable = true)
 |-- 1/25/20: integer (nullable = true)
 |-- 1/26/20: integer (nullable = true)
 |-- 1/27/20: integer (nullable = true)
 |-- 1/28/20: integer (nullable = true)
 |-- 1/29/20: integer (nullable = true)
 |-- 1/30/20: integer (nullable = true)
 |-- 1/31/20: integer (nullable = true)
 |-- 2/1/20: integer (nullable = true)
 |-- 2/2/20: integer (nullable = true)
 |-- 2/3/20: integer (nullable = true)
 |-- 2/4/20: integer (nullable = true)
 |-- 2/5/20: integer (nullable = true)
 |-- 2/6/20: integer (nullable = true)
 |-- 2/7/20: integer (nullable = true)
 |-- 2/8/20: integer (nullable = true)
 |-- 2/9/20: integer (nullable = true)
 |-- 2/10/20: integer (nullable = true)
 |-- 2/11/20: integer (nullable = true)
 

In [19]:
df.select('province_state', 'country_region', 'march23').show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------+-------+
|province_state|country_region|march23|
+--------------+--------------+-------+
|          null|      Thailand|    599|
|          null|         Japan|   1086|
|          null|     Singapore|    455|
|          null|         Nepal|      2|
|          null|      Malaysia|   1306|
+--------------+--------------+-------+
only showing top 5 rows

Let's check the granularity of the data for Switzerland.

In [21]:
# Applying a filter by country
df.filter(df.country_region == "Switzerland").select('province_state', 'country_region', 'march23').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------+-------+
|province_state|country_region|march23|
+--------------+--------------+-------+
|          null|   Switzerland|   7245|
+--------------+--------------+-------+

Other countries have more detailed information.

In [23]:
df.filter(df.country_region == "Australia").select('province_state', 'country_region', 'march23').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------+-------+
|      province_state|country_region|march23|
+--------------------+--------------+-------+
|     New South Wales|     Australia|    533|
|            Victoria|     Australia|    296|
|          Queensland|     Australia|    221|
|     South Australia|     Australia|    100|
|From Diamond Prin...|     Australia|      0|
|   Western Australia|     Australia|    120|
|            Tasmania|     Australia|     22|
|  Northern Territory|     Australia|      3|
|Australian Capita...|     Australia|     19|
+--------------------+--------------+-------+

We can apply an `orderBy` to the `DataFrame` to display the most contaminated places.

In [24]:
df.orderBy(df.march23.desc()).select('province_state', 'country_region', 'march23').show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------+-------+
|province_state|country_region|march23|
+--------------+--------------+-------+
|         Hubei|         China|  67800|
|          null|         Italy|  59138|
|          null|         Spain|  28768|
|          null|       Germany|  24873|
|          null|          Iran|  21638|
|        France|        France|  16018|
|      New York|            US|  15793|
|          null|  Korea, South|   8897|
|          null|   Switzerland|   7245|
|United Kingdom|United Kingdom|   5683|
|   Netherlands|   Netherlands|   4204|
|          null|       Belgium|   3401|
|          null|       Austria|   3244|
|          null|        Norway|   2383|
|    Washington|            US|   1996|
|          null|        Sweden|   1934|
|    New Jersey|            US|   1914|
|    California|            US|   1642|
|          null|      Portugal|   1600|
|          null|        Brazil|   1593|
+--------------+--------------+-------+
only showing top 20 rows